# DB2-Salesforce connector: Tool usage clusters mapping to tools

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'ToolToolClusterAssociation__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"sf2021shitOPmlIiFMLnrudgC6oSX0WV1T",     
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [3]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')
    
import pandas as pd
import time
import datetime

from DB2SalesforceAPI import DB2SalesforceAPI

## Obtain tool information from DB2 

In [4]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/wang159_myrmekes' \
                                               %(sql_login_params['username'], sql_login_params['password']))

classtool_info_df = pd.read_sql_query('select * from cluster_classtool_info', engine)

In [5]:
classtool_info_df.head(3)

,index,toolname,class_id
0,0,geneticalgo,0
1,1,geneticalgo,1
2,2,geneticalgo,2


## Obtain Salesforce IDs

In [6]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [7]:
# query the Salesforce IDs for contacts and citations. when updating junction objects, these IDs must be used

# get Salesforce ID for contacts
sf_toolname_df = db_s.query_data('SELECT Id, Tool_name__c FROM nanoHUB_tools__c where Tool_name__c != NULL')

[Success] Bulk job creation successful. Job ID = 7505w00000Mt5MDAAZ
{"id":"7505w00000Mt5MDAAZ","operation":"query","object":"nanoHUB_tools__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-06-09T01:55:56.000+0000","systemModstamp":"2020-06-09T01:55:56.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000Mt5MDAAZ","operation":"query","object":"nanoHUB_tools__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-06-09T01:55:56.000+0000","systemModstamp":"2020-06-09T01:55:57.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":1594,"retries":0,"totalProcessingTime":322}
[Success] Bulk job completed successfully.


In [8]:
# get Salesforce ID for citations
sf_clusterID_df = db_s.query_data('SELECT Id, ID__c FROM tool_usage_cluster__c')

sf_clusterID_df['ID__c'] = sf_clusterID_df['ID__c'].astype('int')

[Success] Bulk job creation successful. Job ID = 7505w00000Mt5MHAAZ
{"id":"7505w00000Mt5MHAAZ","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-06-09T01:56:08.000+0000","systemModstamp":"2020-06-09T01:56:08.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000Mt5MHAAZ","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-06-09T01:56:08.000+0000","systemModstamp":"2020-06-09T01:56:09.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":3534,"retries":0,"totalProcessingTime":347}
[Success] Bulk job completed successfully.


## Match data with Salesforce format

In [9]:
# Display
display(classtool_info_df.head(2))
display(sf_toolname_df.head(2))
display(sf_clusterID_df.head(2))

,index,toolname,class_id
0,0,geneticalgo,0
1,1,geneticalgo,1


,Id,Tool_name__c
0,a0s5w00000k5MOwAAM,ellipsom
1,a0s5w00000k5MOxAAM,hydrolab


,ID__c,Id
0,0,a0w5w000009Q7CqAAK
1,1,a0w5w000009Q7CrAAK


In [10]:
# merge dataframes
ct_tolink_df = pd.merge(classtool_info_df, sf_toolname_df, how='inner', left_on='toolname', right_on='Tool_name__c')\
                           .rename(columns={'Id':'SF_ID_tool'})

ct_tolink_df = pd.merge(ct_tolink_df, sf_clusterID_df, how='inner', left_on='class_id', right_on='ID__c')\
                           .rename(columns={'Id':'SF_ID_cluster'})

# display
ct_tolink_df.head(2)

,index,toolname,class_id,SF_ID_tool,Tool_name__c,ID__c,SF_ID_cluster
0,0,geneticalgo,0,a0s5w00000k5MXTAA2,geneticalgo,0,a0w5w000009Q7CqAAK
1,1,geneticalgo,1,a0s5w00000k5MXTAA2,geneticalgo,1,a0w5w000009Q7CrAAK


In [11]:
# create a new column for object name
ct_tolink_df['Name'] = ct_tolink_df.apply(lambda x: '%s_%d'%(x.Tool_name__c, x.ID__c), axis=1)

In [12]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Name']         = ct_tolink_df['Name']
df_sf['Tool__c'] = ct_tolink_df['SF_ID_tool']
df_sf['Tool_Usage_Cluster__c'] = ct_tolink_df['SF_ID_cluster']

sf_original_fields = df_sf.columns

# display
display(df_sf.head(2).T)
display('Total rows: ', df_sf.shape[0])

,0,1
Name,geneticalgo_0,geneticalgo_1
Tool__c,a0s5w00000k5MXTAA2,a0s5w00000k5MXTAA2
Tool_Usage_Cluster__c,a0w5w000009Q7CqAAK,a0w5w000009Q7CrAAK


'Total rows: '

5476

## To Salesforce Sales Cloud CRM

In [13]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [14]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000Mt5MMAAZ
hello
[Success] CSV upload successful. Job ID = 7505w00000Mt5MMAAZ
[Success] Closing job successful. Job ID = 7505w00000Mt5MMAAZ


In [15]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_status())

{'apexProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apiVersion': 47.0,
 'columnDelimiter': 'COMMA',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'createdById': '0055w00000ArpYvAAJ',
 'createdDate': '2020-06-09T01:56:21.000+0000',
 'externalIdFieldName': 'Name',
 'id': '7505w00000Mt5MMAAZ',
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'numberRecordsFailed': 0,
 'numberRecordsProcessed': 0,
 'object': 'ToolToolClusterAssociation__c',
 'operation': 'upsert',
 'retries': 0,
 'state': 'UploadComplete',
 'systemModstamp': '2020-06-09T01:56:22.000+0000',
 'totalProcessingTime': 0}
